In [1]:
import os
import time
import certifi
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Usa el bundle de certifi para evitar errores de certificado en requests
os.environ.setdefault("REQUESTS_CA_BUNDLE", certifi.where())
os.environ.setdefault("SSL_CERT_FILE", certifi.where())


'/home/pablo/.local/lib/python3.10/site-packages/certifi/cacert.pem'

In [2]:
VERIFY_SSL = os.getenv("OAI_VERIFY_SSL", "false").lower() == "true"
CA_BUNDLE = os.getenv("OAI_CA_BUNDLE") or certifi.where()

# Pon VERIFY_SSL=True si quieres validar el certificado con CA_BUNDLE; se desactiva por defecto


In [3]:
base_url = catalog.load('params:oai_extract_item_by_col_options.base_url')
context = catalog.load('params:oai_extract_item_by_col_options.context')
env = 'dev'

                    INFO     Loading data from params:oai_extract_item_by_col_options.base_url  ]8;id=49038;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=736114;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:oai_extract_item_by_col_options.context   ]8;id=850136;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=847136;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [4]:
df_set = catalog.load('dv_oai/dim_oai_col')
df_set

                    INFO     Loading data from dv_oai/dim_oai_col (SQLTableDataset)...          ]8;id=886391;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=871132;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,set_id,name,processed,set_hk
0,col_11336_279,Articulos(CCT - SAN LUIS),True,b'\x00\x01\xc4\xd7\xe6\xf9\xe7]\x11\xdb\x13\x9...
1,col_11336_89896,Libros(IFIMAR),False,b'@\xc6\xca\x86w+\xe3D\xde\x85\xd8\xf2\xab\x89...
2,col_11336_90118,Capítulos de libros(CIGEOBIO),False,"b'\t\x94\xbd,\x04Y\xc6\xa0\x18\xb5e\x14<\x85\x..."
3,col_11336_134214,Datos de Investigación(CINDEFI),False,b'\x9a5\x8b~\x94[=8\xe9\xc9\n\xe0$}\xaa\xd3'
4,col_11336_134299,Datos de Investigación(IFIBYNE),False,b'%\xeb\xd8\xff\xdd\x88\x97\xb8N\x1d\x1ec}O{\xa4'
...,...,...,...,...
2068,col_11336_116668,Eventos(IQUIFIB),False,b'\x87\xe7\xa1\x07\xed\xc3\xa5\xc1\x80\xbe\x1d...
2069,col_11336_116578,Eventos(IMBIV),False,b'\xb0\xe5a\x81\x8a\xd1\xdd\x02\xfb7\xd4jS|\x8...
2070,col_11336_116632,Eventos(INIGEM),False,b'\xa5X;\xe3\xc7H\xa6\x08<O\xe1S!\x85CU'
2071,col_11336_90246,Capítulos de libros(IMAM),False,b'\xbf\xfdS\x10\x0eW\x95)\xf3?dJ?\xe9\xc6B'


In [5]:
col_iteration_limit = 5
set_ids = (
    df_set[df_set['processed'] == False]
    .head(col_iteration_limit)
    .iloc[:, 0]
    .tolist()
)
set_ids


['col_11336_89896', 'col_11336_90118', 'col_11336_134214', 'col_11336_134299', 'col_11336_134431']

In [6]:
def get_oai_records(base_url, verify=None):
    start_time = time.time()

    verify_param = CA_BUNDLE if VERIFY_SSL else False
    if verify is not None:
        verify_param = verify

    response = requests.get(base_url, verify=verify_param)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Sleeping for {elapsed_time:.2f} seconds")
    time.sleep(elapsed_time)

    if response.status_code == 200:
        return response
    else:
        print(f"Error: {response.status_code}")
        return None


In [7]:
def oai_extract_item_by_col(base_url: str, context: str, df_set: pd.DataFrame, env: str, col_iteration_limit: int = 1, verify=None) -> pd.DataFrame:
    records = []
    iteration_limit = 2

    pending_cols = df_set[df_set["processed"] == False]
    col_ids = pending_cols.head(col_iteration_limit).iloc[:, 0].tolist()

    if not col_ids:
        print("No se encontraron colecciones pendientes con processed=False.")

    for set_id in col_ids:
        resumption_token = 0
        iteration_count = 0

        while True:
            if env == 'dev' and iteration_count >= iteration_limit:
                break

            params = f'/{context}?verb=ListRecords&resumptionToken=oai_dc///{set_id}/{resumption_token}'
            url = base_url + params

            print(f"Consultando: {url}")

            response = get_oai_records(url, verify=verify)

            resumption_token += 100
            iteration_count += 1

            if not response or not response.ok:
                print(f"Error al consultar: {url}")
                break

            xml_content = response.text
            root = ET.fromstring(xml_content)
            ns = {
                'oai': 'http://www.openarchives.org/OAI/2.0/',
                'dc': 'http://purl.org/dc/elements/1.1/'
            }

            record_nodes = root.findall('.//oai:record', ns)

            if not record_nodes:
                print("No se encontraron más registros.")
                break

            for record in record_nodes:
                identifier = record.find('.//oai:identifier', ns)
                item_id = identifier.text if identifier is not None else None
                metadata = record.find('.//oai:metadata', ns)

                if metadata is None:
                    continue

                # Valores simples
                title = metadata.find('.//dc:title', ns)
                date_issued = metadata.find('.//dc:date', ns)

                # Multivaluados
                creators = [e.text for e in metadata.findall('.//dc:creator', ns)]
                types = [e.text for e in metadata.findall('.//dc:type', ns)]
                identifiers = [e.text for e in metadata.findall('.//dc:identifier', ns)]
                languages = [e.text for e in metadata.findall('.//dc:language', ns)]
                publishers = [e.text for e in metadata.findall('.//dc:publisher', ns)]
                subjects = [e.text for e in metadata.findall('.//dc:subject', ns)]
                relations = [e.text for e in metadata.findall('.//dc:relation', ns)]
                rights = [e.text for e in metadata.findall('.//dc:rights', ns)]

                records.append({
                    'item_id': item_id,
                    'col_id': set_id,
                    'title': title.text if title is not None else None,
                    'date_issued': date_issued.text if date_issued is not None else None,
                    'creators': creators,
                    'types': types,
                    'identifiers': identifiers,
                    'languages': languages,
                    'subjects': subjects,
                    'publishers': publishers,
                    'relations': relations,
                    'rights': rights
                })

    df = pd.DataFrame(records)

    timestamp = pd.Timestamp.now(tz="UTC").normalize()
    df['extract_datetime'] = timestamp
    df['load_datetime'] = timestamp

    return df, df.head(100)


In [8]:
df, df_dev = oai_extract_item_by_col(base_url, context, df_set, env, col_iteration_limit)


Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_89896/0


[11/25/25 12:53:28] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=31076;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=60944;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 11.85 seconds
Error: 500
Error al consultar: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_89896/0
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_90118/0


[11/25/25 12:53:52] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=300643;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=936123;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 14.47 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_90118/100


...

Sleeping for 3.31 seconds
No se encontraron más registros.
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_134431/0


[11/25/25 12:54:30] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=850479;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=17364;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.46 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_134431/100


[11/25/25 12:54:31] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=487768;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=83618;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.19 seconds
No se encontraron más registros.


In [9]:
df

,item_id,col_id,title,date_issued,creators,types,identifiers,languages,subjects,publishers,relations,rights,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/170925,col_11336_90118,"The influence of economics, politics and envir...",2015,"[Bager, Alex, Borghi, Carlos Eduardo, Secco, H...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/170925, Bager, Al...",[eng],"[ecomomic development, impact on biodiversity,...",[John Wiley & Sons Inc.],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/125102,col_11336_90118,The Carboniferous MTD Complex at La Peña Canyo...,2019,"[Valdez Buso, Victoria, Milana, Juan Pablo, So...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/125102, Valdez Bu...",[eng],"[submarine landslides, mass transport deposits...",[Wiley],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
2,oai:ri.conicet.gov.ar:11336/205776,col_11336_90118,Geophysical Methods Applied to the Study of La...,2021,"[Prezzi, Claudia Beatriz, Orgeira, Maria Julia...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/205776, Prezzi, C...",[eng],"[Gravity Magnetic and resistivity surveys, Lak...",[Springer],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
3,oai:ri.conicet.gov.ar:11336/156736,col_11336_90118,La avifauna de Agua Negra,2013,"[Ortiz, Grisel, Gómez, Héctor A., Blanco Fager...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/156736, Ortiz, Gr...",[spa],"[AVIFAUNA, AGUA NEGRA, https://purl.org/becyt/...",[Universidad Nacional de San Juan],[],"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
4,oai:ri.conicet.gov.ar:11336/125106,col_11336_90118,"Block Generation, Deformation, and Interaction...",2019,"[Sobiesiak, Matheus, Valdez Buso, Victoria, Kn...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/125106, Sobiesiak...",[eng],"[Submarine landslides, mass transprot deposit,...",[Wiley],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
5,oai:ri.conicet.gov.ar:11336/156860,col_11336_90118,Consideraciones sobre la gestión del patrimoni...,2013,"[García, Eduardo Alejandro, Martinez Carretero...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/156860, García, E...",[spa],"[Patrimonio cultural, Recursos culturales, Rec...",[Universidad Nacional de San Juan],[info:eu-repo/semantics/reference/url/https://...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
6,oai:ri.conicet.gov.ar:11336/249151,col_11336_90118,Interweaving Scientific and Local Ecological K...,2023,"[Campos, Claudia Monica, Moreno, María Carolin...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/249151, Campos, C...",[eng],"[COGNITIVE SALIENCE, CULTURAL SALIENCE, ECOLOG...",[Springer],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
7,oai:ri.conicet.gov.ar:11336/173762,col_11336_90118,Dendrochronological Study of the Xeric and Mes...,2020,"[Hadad, Martín Ariel, Arco Molina, Julieta Gab...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/173762, Hadad, Ma...",[eng],"[DENDROECOLOGY, SOUTH AMERICA, FOREST CONSERVA...",[Springer Nature Switzerland AG],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-1

In [10]:
df_dev

,item_id,col_id,title,date_issued,creators,types,identifiers,languages,subjects,publishers,relations,rights,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/170925,col_11336_90118,"The influence of economics, politics and envir...",2015,"[Bager, Alex, Borghi, Carlos Eduardo, Secco, H...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/170925, Bager, Al...",[eng],"[ecomomic development, impact on biodiversity,...",[John Wiley & Sons Inc.],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/125102,col_11336_90118,The Carboniferous MTD Complex at La Peña Canyo...,2019,"[Valdez Buso, Victoria, Milana, Juan Pablo, So...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/125102, Valdez Bu...",[eng],"[submarine landslides, mass transport deposits...",[Wiley],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
2,oai:ri.conicet.gov.ar:11336/205776,col_11336_90118,Geophysical Methods Applied to the Study of La...,2021,"[Prezzi, Claudia Beatriz, Orgeira, Maria Julia...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/205776, Prezzi, C...",[eng],"[Gravity Magnetic and resistivity surveys, Lak...",[Springer],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
3,oai:ri.conicet.gov.ar:11336/156736,col_11336_90118,La avifauna de Agua Negra,2013,"[Ortiz, Grisel, Gómez, Héctor A., Blanco Fager...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/156736, Ortiz, Gr...",[spa],"[AVIFAUNA, AGUA NEGRA, https://purl.org/becyt/...",[Universidad Nacional de San Juan],[],"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
4,oai:ri.conicet.gov.ar:11336/125106,col_11336_90118,"Block Generation, Deformation, and Interaction...",2019,"[Sobiesiak, Matheus, Valdez Buso, Victoria, Kn...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/125106, Sobiesiak...",[eng],"[Submarine landslides, mass transprot deposit,...",[Wiley],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
5,oai:ri.conicet.gov.ar:11336/156860,col_11336_90118,Consideraciones sobre la gestión del patrimoni...,2013,"[García, Eduardo Alejandro, Martinez Carretero...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/156860, García, E...",[spa],"[Patrimonio cultural, Recursos culturales, Rec...",[Universidad Nacional de San Juan],[info:eu-repo/semantics/reference/url/https://...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
6,oai:ri.conicet.gov.ar:11336/249151,col_11336_90118,Interweaving Scientific and Local Ecological K...,2023,"[Campos, Claudia Monica, Moreno, María Carolin...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/249151, Campos, C...",[eng],"[COGNITIVE SALIENCE, CULTURAL SALIENCE, ECOLOG...",[Springer],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
7,oai:ri.conicet.gov.ar:11336/173762,col_11336_90118,Dendrochronological Study of the Xeric and Mes...,2020,"[Hadad, Martín Ariel, Arco Molina, Julieta Gab...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/173762, Hadad, Ma...",[eng],"[DENDROECOLOGY, SOUTH AMERICA, FOREST CONSERVA...",[Springer Nature Switzerland AG],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-1

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   item_id           24 non-null     object             
 1   col_id            24 non-null     object             
 2   title             24 non-null     object             
 3   date_issued       24 non-null     object             
 4   creators          24 non-null     object             
 5   types             24 non-null     object             
 6   identifiers       24 non-null     object             
 7   languages         24 non-null     object             
 8   subjects          24 non-null     object             
 9   publishers        24 non-null     object             
 10  relations         24 non-null     object             
 11  rights            24 non-null     object             
 12  extract_datetime  24 non-null     datetime64[us, UTC]
 13  load_da